In [11]:
import numpy as np
import tifffile
from scipy import ndimage
from skimage.filters import threshold_otsu
import matplotlib.pyplot as plt
from skimage.measure import label, regionprops


file_name = f"/Users/demir/Documents/Hva_AI/afstudeerproject/codes_yunus/tif_deneme_1.tif"

img = tifffile.imread(file_name)  # (Z, C, Y, X)
print(img.shape)

# Sadece 4. kanalı (C3, index 3) tüm Z-stack ile al
channel_index = 3
c3_stack = img[:, channel_index, :, :]  # Shape: (9, 1024, 1024)

print(c3_stack.shape)


(9, 4, 1024, 1024)
(9, 1024, 1024)


In [12]:
from cellpose import models, io
import tifffile as tiff
import matplotlib.pyplot as plt

# Load your 3D image (TIF stack)
img_3d = tiff.imread(c3_stack)  # Shape: (Z, Y, X)

# Load Cellpose model for nuclei or cytoplasm
model = models.Cellpose(model_type='nuclei')  # or 'cyto'

# Run segmentation in 3D mode
masks, flows, styles, diams = model.eval(
    img_3d,
    channels=[0, 0],    # change depending on the channel (e.g., [0, 0] for grayscale)
    diameter=None,      # or specify a value like 17
    do_3D=True,         # IMPORTANT: enable 3D segmentation
    anisotropy=1.0      # change if Z spacing ≠ XY spacing
)

# Save result as a labeled TIFF
tiff.imwrite('segmented_masks.tif', masks.astype('uint16'))

# Optional: visualize one Z-slice
plt.imshow(img_3d[img_3d.shape[0] // 2], cmap='gray')
plt.contour(masks[img_3d.shape[0] // 2], colors='r', linewidths=0.5)
plt.title("Middle Z-slice with segmentation")
plt.show()


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()